In [ ]:

import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

PARTICIPANT = "5003_I_board_one"
KEYPOINTS = ["right_wrist", "left_wrist"]  # Add or remove keypoints as needed


In [ ]:

# Load keypoint data
file_path = '5003_I_board_one_keypoints.csv'  # Update this path to your actual file path
keypoints_df = pd.read_csv(file_path)
keypoints_df = pd.DataFrame(keypoints_df)
keypoints_df.head()


In [ ]:

# Initialize an empty DataFrame to store final results
final_df = pd.DataFrame({'time_ms': keypoints_df['time_ms']})

# Loop through each keypoint specified in the KEYPOINTS list
for keypoint in KEYPOINTS:
    # Extracting original x and y coordinates
    x_original = keypoints_df[f'{keypoint}_x'].values
    y_original = keypoints_df[f'{keypoint}_y'].values
    time_original = keypoints_df['time_ms'].values

    # Interpolation for upsampling
    time_upsampled = np.linspace(time_original[0], time_original[-1], len(time_original) * 10)
    interp_x = interp1d(time_original, x_original, kind='linear')
    interp_y = interp1d(time_original, y_original, kind='linear')
    x_upsampled = interp_x(time_upsampled)
    y_upsampled = interp_y(time_upsampled)

    # Smoothing using Savitzky-Golay filter
    window_length = 51  # Choose an appropriate window length
    polyorder = 3  # Choose an appropriate polynomial order
    x_smooth = savgol_filter(x_upsampled, window_length, polyorder)
    y_smooth = savgol_filter(y_upsampled, window_length, polyorder)

    # Speed calculation
    delta_time = np.diff(time_upsampled)
    speed_x = np.diff(x_smooth) / delta_time
    speed_y = np.diff(y_smooth) / delta_time
    speed = np.sqrt(speed_x ** 2 + speed_y ** 2)

    # Add to the final DataFrame
    temp_df = pd.DataFrame({
        'time_ms': time_upsampled[:-1],
        f'{keypoint}_x': x_smooth[:-1],
        f'{keypoint}_y': y_smooth[:-1],
        f'{keypoint}_speed': speed
    })
    final_df = pd.merge(final_df, temp_df, on='time_ms', how='right')

final_df.head()
